In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

import sklearn
from sklearn.inspection import permutation_importance

from sklearn.utils import class_weight

import os
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

from keras.utils import plot_model

from datetime import datetime
from datetime import timedelta

#!pip install seaborn
#import seaborn as sns

In [2]:
def prepare_x_y_data(input_csv):
    # Read in formatted data
    data = pd.read_csv(input_csv, index_col = False)
    data = data.fillna(0)
    
    ### Delete unneeded columns - we currently include data from all sensors (even incomplete ones)
    sensor_ids = data['sensor_id']
    #data = data.drop(['sensor_id'],axis=1) # don't want this included
    # Get rid of columns in which none of the sensors have a value
    for column in data.columns:
        if np.nanmax(data[column]) ==0:
            del data[column]
            
    # Filter columns using the regex pattern in function input
    regex_pattern = 'buildings$|street_inf$|landmarks$'
    data = data[data.columns.drop(list(data.filter(regex=regex_pattern)))].copy()
    
    ### Add a random variable (to compare performance of other variables against)
    rng = np.random.RandomState(seed=42)
    data['random'] = np.random.random(size=len(data))
    data["random_cat"] = rng.randint(3, size=data.shape[0])
    
    ## Prepare data for modelling 
    ### Split into predictor/predictand variables
    Xfull = data.drop(['hourly_counts'], axis =1)
    Yfull = data['hourly_counts'].values
       
    ### Store the (non Sin/Cos) time columns and then remove them (Need them later to segment the results by hour of the day)
    data_time_columns = Xfull[['day_of_month_num', 'time', 'weekday_num', 'time_of_day', 'datetime']]
    #Xfull = Xfull.drop(['day_of_month_num', 'time', 'weekday_num', 'time_of_day','datetime', 'month_num'],axis=1)
    Xfull = Xfull.drop(['day_of_month_num', 'time', 'weekday_num', 'time_of_day', 'month_num'],axis=1)
    return Xfull, Yfull, data_time_columns

In [3]:
def normalize(df, target_column):
    
    result = df.copy()
    
    for feature_name in df.columns:
        
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        
        if feature_name == 'footfall':
            result['footfall_norm'] = (df[feature_name] - min_value) / (max_value - min_value)
            
        else:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    
    cols = list(result.columns)
    column_list = cols[:-2] + cols[-1:] + cols[-2:-1]
    result = result[column_list]
    
    return result

In [4]:
buffer_size_m = 400
input_csv = "formatted_data_for_modelling_allsensors_{}.csv".format(buffer_size_m)

X_data, Y_data, data_time_columns = prepare_x_y_data(input_csv)

X_data = X_data.iloc[:2198889]
Y_data = Y_data[:2198889]
data_time_columns = data_time_columns.iloc[:2198889]

In [5]:
df = X_data.copy()

df = df.loc[:,['datetime', 'Temp', 'Humidity' ,'Pressure', 'Rain', 'WindSpeed', 'Rainfall amount (millimetres)',
               'Sin_time','Cos_time','Sin_month_num','Cos_month_num','Sin_weekday_num','Cos_weekday_num']] 

df['footfall'] = Y_data

In [6]:
city_df = df.iloc[:,:-1].drop_duplicates(keep='first').reset_index(drop=True)
footfall = list(df.loc[:, ['datetime', 'footfall']].groupby(['datetime']).sum()['footfall'])
city_df['footfall'] = footfall

In [7]:
city_df

,datetime,Temp,Humidity,Pressure,Rain,WindSpeed,Rainfall amount (millimetres),Sin_time,Cos_time,Sin_month_num,Cos_month_num,Sin_weekday_num,Cos_weekday_num,footfall
0,2011-01-01 00:00:00,23.0,57.0,1008.0,0,6.0,0.0,0.000000e+00,1.000000,0.5,8.660254e-01,-0.781831,0.623490,5880
1,2011-01-01 01:00:00,21.5,62.5,1007.5,0,6.0,0.0,2.697968e-01,0.962917,0.5,8.660254e-01,-0.781831,0.623490,8111
2,2011-01-01 02:00:00,21.0,64.0,1007.0,0,0.0,0.0,5.195840e-01,0.854419,0.5,8.660254e-01,-0.781831,0.623490,10310
3,2011-01-01 03:00:00,20.0,69.0,1007.0,0,0.0,0.0,7.308360e-01,0.682553,0.5,8.660254e-01,-0.781831,0.623490,8079
4,2011-01-01 04:00:00,17.0,80.5,1008.0,0,8.0,0.0,8.878852e-01,0.460065,0.5,8.660254e-01,-0.781831,0.623490,3774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71894,2019-03-15 19:00:00,18.0,73.0,1019.0,0,16.5,0.0,-8.878852e-01,0.460065,1.0,6.123234e-17,-0.974928,-0.222521,37028
71895,2019-03-15 20:00:00,17.0,73.0,1020.5,0,14.5,0.0,-7.308360e-01,0.682553,1.0,6.123234e-17,-0.974928,-0.222521,31003
71896,2019-03-15 21:00:00,17.0,77.0,1021.0,0,12.5,0.0,-5.195840e-01,0.854419,1.0,6.123234e-17,-0.974928,-0.222521,30757
71897,2019-03-15 22:00:00,17.0,77.0,1021.0,0,10.0,0.0,-2.697968e-01,0.962917,1.0,6.123234e-17,-0.974928,-0.222521,28876


In [8]:
datetimes = city_df['datetime']
for i in range(len(datetimes)-1):
    if datetime.strptime(datetimes[i+1], "%Y-%m-%d %H:%M:%S") != datetime.strptime(datetimes[i], "%Y-%m-%d %H:%M:%S") + timedelta(hours=1):
        print(datetimes[i], i)


2013-10-06 01:00:00 24217
2014-10-05 01:00:00 32952
2015-10-04 01:00:00 41687
2016-10-02 01:00:00 50422
2017-10-01 01:00:00 59157


In [9]:
footfall_max = city_df['footfall'].max()
footfall_min = city_df['footfall'].min()

In [10]:
city_df = city_df.drop(columns = ['datetime'])

city_df = city_df.drop(columns = ['Temp', 'Humidity' ,'Pressure', 'Rain', 'WindSpeed', 'Rainfall amount (millimetres)',
                                   'Sin_time','Cos_time','Sin_month_num','Cos_month_num','Sin_weekday_num','Cos_weekday_num'])

n=len(city_df)

train_df = city_df[0:int(n*0.7)]
val_df = city_df[int(n*0.7):int(n*0.9)]
test_df = city_df[int(n*0.9):]

In [11]:
num_features = city_df.shape[1]

In [12]:
#train_df = normalize(train_df, 'footfall')
#val_df = normalize(val_df, 'footfall')
#test_df = normalize(test_df, 'footfall')

In [13]:
train_df = train_df.dropna(axis='columns')
val_df = val_df.dropna(axis='columns')
test_df = test_df.dropna(axis='columns')

In [14]:
train_df

,footfall
0,5880
1,8111
2,10310
3,8079
4,3774
...,...
50324,7302
50325,3110
50326,1490
50327,920


In [15]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
    
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        
        self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}
    
        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
    
        self.total_window_size = input_width + shift
    
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
    
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
        
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        #print(inputs.shape)
        #inputs[:,:,-1] = (inputs[:,:,-1] - footfall_min) / (footfall_max - footfall_min)

        
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)
    
        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])
    
        return inputs, labels

In [16]:
w2 = WindowGenerator(input_width=6, label_width=1, shift=1, label_columns=['footfall'])

In [17]:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])

2024-01-10 11:55:20.138855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 11:55:21.416358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 59 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:0b:00.0, compute capability: 7.0
2024-01-10 11:55:21.418751: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30436 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:85:00.0, compute capability: 7.0


In [18]:
example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 7, 1)
Inputs shape: (3, 6, 1)
Labels shape: (3, 1, 1)


In [19]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)
    ds = ds.map(self.split_window)
    return ds

WindowGenerator.make_dataset = make_dataset

In [20]:
def plot(self, model=None, plot_col='footfall', max_subplots=10):
    
    inputs, labels = self.example
    plt.figure(figsize=(12, 18))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    print(max_n)
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index-1],
                 label='Inputs', marker='.', zorder=-10)
    
        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index
    
        if label_col_index is None:
            continue
    
        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            plt.scatter(self.label_indices, predictions[n,label_col_index],
                      marker='X', edgecolors='k', label='Predictions',
                      c='#ff7f0e', s=64)
    
        if n == 0:
            plt.legend()
    
    plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [21]:
@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [22]:
MAX_EPOCHS = 10_000

def compile_and_fit(model, window, patience=3):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs = MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping])
    return history

In [23]:
# Each element is an (inputs, label) pair.
w2.train.element_spec

(TensorSpec(shape=(None, 6, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [24]:
for example_inputs, example_labels in w2.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 6, 1)
Labels shape (batch, time, features): (32, 1, 1)


In [135]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [136]:
print('Input shape:', w2.example[0].shape)
print('Output shape:', lstm_model(w2.example[0]).shape)

Input shape: (32, 6, 1)
Output shape: (32, 1)


In [137]:
val_performance = {}
performance = {}

In [138]:
history = compile_and_fit(lstm_model, w2)

Epoch 1/10000
1573/1573 [==============================] - 8s 4ms/step - loss: 220000800.0000 - mean_absolute_error: 11780.0527 - val_loss: 431089120.0000 - val_mean_absolute_error: 17604.3789
Epoch 2/10000
1573/1573 [==============================] - 5s 3ms/step - loss: 219090224.0000 - mean_absolute_error: 11741.3291 - val_loss: 429834880.0000 - val_mean_absolute_error: 17568.7109
Epoch 3/10000
1573/1573 [==============================] - 6s 4ms/step - loss: 218272800.0000 - mean_absolute_error: 11706.4863 - val_loss: 428625600.0000 - val_mean_absolute_error: 17534.2441
Epoch 4/10000
1573/1573 [==============================] - 6s 3ms/step - loss: 217469968.0000 - mean_absolute_error: 11672.1475 - val_loss: 427424256.0000 - val_mean_absolute_error: 17499.9531
Epoch 5/10000
1573/1573 [==============================] - 6s 4ms/step - loss: 216671360.0000 - mean_absolute_error: 11638.0059 - val_loss: 426227296.0000 - val_mean_absolute_error: 17465.7070
Epoch 6/10000
1573/1573 [==========

In [ ]:
val_performance['LSTM'] = lstm_model.evaluate(w2.val)
performance['LSTM'] = lstm_model.evaluate(w2.test, verbose=1)

In [ ]:
model.save('whole_city_rnn')

In [25]:
val_performance = {}
performance = {}

lstm_model = tf.keras.models.load_model('sensor_54_window_6_model')

val_performance['LSTM'] = lstm_model.evaluate(w2.val)
performance['LSTM'] = lstm_model.evaluate(w2.test, verbose=1)

2024-01-10 11:55:24.433969: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2024-01-10 11:55:24.493547: E tensorflow/stream_executor/dnn.cc:868] CUDNN_STATUS_INTERNAL_ERROR
in tensorflow/stream_executor/cuda/cuda_dnn.cc(2040): 'cudnnRNNForwardTraining( cudnn.handle(), rnn_desc.handle(), model_dims.max_seq_length, input_desc.handles(), input_data.opaque(), input_h_desc.handle(), input_h_data.opaque(), input_c_desc.handle(), input_c_data.opaque(), rnn_desc.params_handle(), params.opaque(), output_desc.handles(), output_data->opaque(), output_h_desc.handle(), output_h_data->opaque(), output_c_desc.handle(), output_c_data->opaque(), workspace.opaque(), workspace.size(), reserve_space.opaque(), reserve_space.size())'
2024-01-10 11:55:24.493613: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1563 : INTERNAL: Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers

InternalError: Graph execution error:

Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 1, 32, 1, 6, 32, 32] 
	 [[{{node CudnnRNN}}]]
	 [[sequential_3/lstm_3/PartitionedCall]] [Op:__inference_test_function_7513]